In [113]:
import numpy as np
import math
import plumed
import mdtraj as mdj
import matplotlib.pyplot as plt
import pandas as pd 

In [114]:
# make fake topology
def make_top(num_atoms):
    data = []
    for i in range(num_atoms):
        data.append(dict(serial=i, name="H", element="H",
            resSeq=i + 1, resName="UNK", chainID=0))

    data = pd.DataFrame(data)

    topology = mdj.Topology.from_dataframe(data, bonds=np.zeros((0, 2), dtype='int'))
    return topology

def make_funnel_traj(positions, unitcell_lengths, unitcell_angles):
    n_atoms = positions.shape[1]
    n_frames = positions.shape[0]
    
    topology = make_top(n_atoms)

    new_traj = mdj.Trajectory(xyz=positions, topology=topology, 
                unitcell_lengths=unitcell_lengths,
                unitcell_angles=unitcell_angles)
    return new_traj
    
    



In [115]:
def makeFunnel(p1_com, p2_com, s_cent=1.6, beta_cent=0.3, wall_width=1.5, wall_buffer=1.5, lower_wall=0.5, 
               upper_wall=3.7, vec_step=0.25, angle_sample=20):
    
    s_cent = float(s_cent)
    beta_cent = float(beta_cent)
    wall_width = float(wall_width)
    wall_buffer = float(wall_buffer)
    lower_wall = float(lower_wall)
    upper_wall = float(upper_wall)
    vec_step = float(vec_step)
    angle_sample = float(angle_sample)

    # get coords of the origin and vector points
    #origin = cmd.get_coords(selection, 1)[0]
    origin = p1_com
    v1 = p1_com
    v2 = p2_com
    # calculate the vector defined by points p1 and p2
    vec = np.array(v2, dtype=float) - np.array(v1, dtype=float)
    # BEWARE: inconsistency with linalg, if vec is a list and not an array!!!
#    print(np.linalg.norm(vec), np.linalg.norm(v2 - v1))
    # make it a unit vector
    unit_vec = vec/np.linalg.norm(vec)
#    print(np.linalg.norm(vec), vec, np.linalg.norm(unit_vec), unit_vec)
    # how to get orthogonal vectors
    # https://math.stackexchange.com/questions/133177/finding-a-unit-vector-perpendicular-to-another-vector
    # determine 1st orthogonal vector
    a0 = np.random.randint(1,10)
    a1 = np.random.randint(1,10)
    a2 = -(a0*vec[0] + a1*vec[1])/vec[2]
    a = np.asarray([a0, a1, a2])
    unit_a = a/np.linalg.norm(a)
    # determine 2nd orthogonal vector
    unit_b = np.cross(unit_a, unit_vec)
#    print(unit_vec, unit_a, unit_b)
#    print(np.linalg.norm(unit_vec), np.linalg.norm(unit_a), np.linalg.norm(unit_b))
    # iterate along the selected vector
    funnel_coords = []
    for step in np.arange(lower_wall, upper_wall, vec_step):
        # iterate around a circle with its radius defined by the sigmoid function
        radius = (wall_width / (1 + np.exp(beta_cent * (step - s_cent)))) + wall_buffer
        for angle in np.arange(-np.pi, np.pi, 2 * np.pi / angle_sample):
            # calculate parametric functions for this specific case
            # https://math.stackexchange.com/questions/73237/parametric-equation-of-a-circle-in-3d-space
            # generate pseudoatoms along the axis
            pos = origin + unit_vec*step + radius*(np.cos(angle)*unit_a + np.sin(angle)*unit_b)
            funnel_coords.append(pos)
    return funnel_coords

In [116]:
s_cent = 2.3                                    # INFLEXION
beta_cent = 1.0                                 # STEEPNESS
wall_width = 0.85                               # WIDTH (h)
wall_buffer = 0.15                               # BUFFER (f, total width = WIDTH + BUFFER)
lwall = 0.5
uwall = 3.6 
#2.0 1.5 0.6 0.15 0.5 4.0

In [133]:
%%bash
plumed driver --plumed ../outputs/plumed.dat --mf_dcd ../outputs/dist_traj.dcd --pdb ../inputs/protein_ligand.pdb > out.txt

In [134]:

top = make_top(2)
funnel_points = mdj.load_xyz('funnel_points.xyz', top)

In [135]:
traj_funnel_coords = []
for points in funnel_points.xyz:
    #print(points[0], points[1])
    funnel_coords = makeFunnel(points[0], points[1], s_cent, beta_cent, wall_width, wall_buffer, lwall, uwall)
    traj_funnel_coords.append(funnel_coords)
    
traj_funnel_coords = np.array(traj_funnel_coords)
print(traj_funnel_coords.shape)

(4887, 260, 3)


In [136]:
traj = mdj.load_dcd('../outputs/dist_traj.dcd', '../inputs/protein_ligand.pdb')
traj = make_funnel_traj(traj_funnel_coords, traj.unitcell_lengths, traj.unitcell_angles)
traj.save_xyz('../outputs/funnel.xyz')

/home/nz/anaconda3/envs/plumed/lib/python3.8/site-packages/mdtraj/formats/pdb/pdbstructure.py:435: UserWarning: WARNING: two consecutive residues with same number (ATOM    688  N   ARG A  65      41.030  38.590  50.160  1.00  1.00           N  , ATOM    687  O   VAL A  65      40.240  38.830  52.260  1.00  1.00           O  )
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (atom, self._current_residue.atoms[-1]))
/home/nz/anaconda3/envs/plumed/lib/python3.8/site-packages/mdtraj/formats/pdb/pdbstructure.py:435: UserWarning: WARNING: two consecutive residues with same number (ATOM   2374  N   TYR A 184      43.370  38.830  21.130  1.00  1.00           N  , ATOM   2373  O   GLY A 184      42.110  40.660  20.840  1.00  1.00           O  )
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (atom, self._current_residue.atoms[-1]))
/home/nz/anaconda3/envs/plumed/lib/python3.8/site-packages/mdtraj/formats/pdb/pdbstructure.py:435: Use